In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import gym
import wandb
from rl.dqn import DQN

import torch
import torch.nn as nn
import torch.optim as optim

# displaying recordings
from pyvirtualdisplay import Display
import io
import base64
from gym import wrappers
from IPython.display import HTML

In [3]:
display = Display(visible=0, size=(600, 600))
display.start()

In [4]:
MONITOR_DIR = "./gym-results"

env = gym.make("CartPole-v0")
env = wrappers.Monitor(env, MONITOR_DIR, video_callable=lambda ep: True, force=True)

In [5]:
def build_model(in_dim, out_dim):
    network = nn.Sequential(
        nn.Linear(in_dim, 256),
        nn.ELU(),
        nn.Linear(256, 256),
        nn.ELU(),
        nn.Linear(256, 256),
        nn.ELU(),
        nn.Linear(256, out_dim),
    )
    
    return network

In [6]:
adam_opts = {
    "lr": 0.01
}

In [7]:
def post_train_hook(obj, variables):
    wandb.log({
        "loss": variables["loss"],
        "step": variables["steps"],
    })
    
def post_episode_hook(obj, variables, state):
    ep = variables["ep"]
    ret = variables["ep_return"]
    
    if ret >= 195 and not state.get("saved", False):
        if state.get("saved_file", None) is None:
            state["saved_file"] = (
                f"{MONITOR_DIR}/"
                f"openaigym.video.{env.file_infix}.video{ep:06d}.mp4"
            )
        else:
            state["saved"] = True
            wandb.log({
                f"{wandb.run.name}-ep-{ep}": wandb.Video(state["saved_file"])
            })
    
    wandb.log({
        "episode": ep,
        "return": ret,
        "avg_return": obj.return_history.mean(),
        "max_return": obj.return_history.max(),
    })
    

In [8]:
dqn = DQN(env, build_model, optim.Adam, adam_opts)

[env]: num_actions=2
[env]: obs_dim=4


In [9]:
config = {
    "adam_opts": adam_opts
}

In [10]:
wandb.init(project="deep-q-network", config=config);
wandb.watch(dqn.model, log="all")

Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Currently logged in as: mattfeng (use `wandb login --relogin` to force relogin)


In [11]:
state = {}

dqn.train(
    post_train_hook=post_train_hook,
    post_episode_hook=lambda a, b: post_episode_hook(a, b, state)
)

[opt][step 1], still warming up.
[opt][step 2], still warming up.
[opt][step 3], still warming up.
[opt][step 4], still warming up.
[opt][step 5], still warming up.
[opt][step 6], still warming up.
[opt][step 7], still warming up.
[opt][step 8], still warming up.
[opt][step 9], still warming up.
[opt][step 10], still warming up.
[opt][step 11], still warming up.
[opt][step 12], still warming up.
[opt][step 13], still warming up.
[opt][step 14], still warming up.
[opt][step 15], still warming up.
[opt][step 16], still warming up.
[opt][step 17], still warming up.
[opt][step 18], still warming up.
[opt][step 19], still warming up.
[opt][step 20], still warming up.
[opt][step 21], still warming up.
[opt][step 22], still warming up.
[opt][step 23], still warming up.
[opt][step 24], still warming up.
[opt][step 25], still warming up.
[opt][step 26], still warming up.
[opt][step 27], still warming up.
[opt][step 28], still warming up.
[opt][step 29], still warming up.
[opt][step 30], still w

KeyboardInterrupt: 